In [1]:
def df_par(df, func):
  """
  Parallel implementation. Does work in Windows.
  Only works in Jupyter Notebook and Linux.
  """
  df_split = np.array_split(df, mp.cpu_count())
  pool = mp.Pool(mp.cpu_count())
  df = pd.concat(pool.map(func, df_split))
  pool.close()
  del df_split
  return df

In [2]:
# load necessary libraries
import os
import datetime
import holidays
import pandas as pd
import numpy as np
import progressbar
import multiprocessing as mp

In [3]:
data_dir = os.path.join('..','data-wrangling','data','flight_data')
# Before importing data, let's define datatypes of columns
types = {0: 'int8', 1: 'category', 2: 'category', 3: 'int16', 4: 'int16', 5: 'int32',
         6: 'int16', 7: 'int32', 18: 'bool', 19: 'int8', 20: 'bool', 24: 'int16',
         25: 'int16', 26: 'int16', 27: 'int16', 28: 'int16', 29: 'int16', 31: 'float32',
         32: 'float32', 33: 'float32', 34: 'float32', 35: 'float32', 36: 'float32', 
         37: 'float32', 38: 'float32', 39: 'float32', 40: 'float32', 41: 'float32', 
         42: 'float32', 43: 'float32', 44: 'float32'}
# Now, import the data
flights = pd.read_csv(os.path.join(data_dir,'2018_final.csv'), usecols=range(0,45), dtype=types, parse_dates=[8, 9, 12, 13, 15, 16])

In [4]:
# Now, we need assign timedelta type to the columns listed below (in parallel)
cols = ['DepDelay', 'TxO', 'TxI', 'ArrDelay', 'ScElaTime', 'AcElaTime', 'AirTime', 'TimeZoneDiff']
for name in cols:
  flights.loc[:, name] = df_par(flights.loc[:, name], pd.to_timedelta)

In [5]:
flights.head(5)

,WeekDay,IATA,TailNum,FlightNum,OrgAirID,OrgMarID,DestAirID,DestMarID,ScDepTime,DepTime,...,Dir_O,WindSp_O,Visib_O,Temp_D,DewPoint_D,RelHum_D,HeatInd_D,Dir_D,WindSp_D,Visib_D
0,1,AA,N156UW,430,14771,32457,11057,31057,2018-01-01 00:20:00,2018-01-01 00:14:00,...,200.0,3.45,8.0,25.000000,-6.0,24.889999,16.250000,20.0,8.05,10.0
1,1,DL,N669DN,806,14771,32457,13487,31650,2018-01-01 00:20:00,2018-01-01 00:14:00,...,200.0,3.45,8.0,-11.000000,-17.0,73.940002,-29.950001,300.0,10.35,10.0
2,1,UA,N14240,1104,10299,30299,11292,30325,2018-01-01 00:25:00,2018-01-01 00:12:00,...,350.0,10.35,10.0,8.100000,3.9,82.580002,-4.350000,150.0,8.05,8.0
3,1,AS,N530AS,174,10299,30299,12889,32211,2018-01-01 00:25:00,2018-01-01 00:12:00,...,350.0,10.35,10.0,46.900002,12.0,24.030001,44.090000,220.0,5.75,10.0
4,1,UA,N66808,2393,14747,30559,12266,31453,2018-01-01 00:27:00,2018-01-01 00:15:00,...,10.0,14.95,10.0,37.900002,28.9,69.730003,29.920000,20.0,12.65,10.0


To start with feature engineering, it would be good to begin with the data we have first without extracting any features. Since our target variable is binary, which is whether a flight is delayed or not, only certain columns that are/might be known prior to scheduled departure time will be selected. These columns are ScDepTime, IATA, TailNum, OrgAirID, DestAirID, and Weather data at origin airport.

In [6]:
flights = flights[(flights.Cncl==0) & (flights.Div==0)]
flights = flights.sort_values('DepTime')
flights.reset_index(drop=True, inplace=True)

In [7]:
feat = pd.DataFrame({'month': flights.ScDepTime.dt.month, 'weekDay': flights.WeekDay, 'dayofMonth': flights.ScDepTime.dt.day,
                     'dayofYear': flights.ScDepTime.dt.dayofyear, 'hour': flights.ScDepTime.dt.hour, 'iata': flights.IATA, 'tailNum': flights.TailNum,
                     'orgAirport': flights.OrgAirID, 'destAirport': flights.DestAirID, 'temp': flights.Temp_O, 'dewPoint': flights.DewPoint_O,
                     'relHum': flights.RelHum_O, 'heatInd': flights.HeatInd_O, 'windDir': flights.Dir_O, 'windSp': flights.WindSp_O,
                     'visib': flights.Visib_O, 'delayed': np.where(flights.DepDelay > pd.Timedelta('0 min'), 1, 0)})
feat.head()

,month,weekDay,dayofMonth,dayofYear,hour,iata,tailNum,orgAirport,destAirport,temp,dewPoint,relHum,heatInd,windDir,windSp,visib,delayed
0,1,1,1,1,0,UA,N14240,10299,11292,26.100000,19.0,74.160004,16.180000,350.0,10.35,10.0,0
1,1,1,1,1,0,AS,N530AS,10299,12889,26.100000,19.0,74.160004,16.180000,350.0,10.35,10.0,0
2,1,1,1,1,0,AA,N156UW,14771,11057,57.000000,48.0,71.809998,57.000000,200.0,3.45,8.0,0
3,1,1,1,1,0,DL,N669DN,14771,13487,57.000000,48.0,71.809998,57.000000,200.0,3.45,8.0,0
4,1,1,1,1,0,UA,N66808,14747,12266,37.900002,30.9,75.639999,29.129999,10.0,14.95,10.0,0


From exploratory data analysis, we have seen that number of flights is also another factor, so we are extracting this feature from our dataframe.

In [8]:
lookup = flights.groupby(['OrgAirID', flights.ScDepTime.dt.dayofyear, flights.ScDepTime.dt.hour]).count().iloc[:, 1].to_frame()
lookup.index.names = ['OrgAirport', 'DayofYear', 'Hour']
lookup.columns = ['Flights']
lookup.head()

Flights
OrgAirport DayofYear Hour         
10135      1         6           2
                     9           2
                     15          1
                     20          1
           2         6           3

In [9]:
temp = pd.DataFrame({'OrgAirport': flights.OrgAirID, 'DayofYear': flights.ScDepTime.dt.dayofyear, 'Hour': flights.ScDepTime.dt.hour})
temp.head()

,OrgAirport,DayofYear,Hour
0,10299,1,0
1,10299,1,0
2,14771,1,0
3,14771,1,0
4,14747,1,0


In [10]:
temp.merge(lookup, left_on=['OrgAirport', 'DayofYear', 'Hour'], right_index=True).tail(10)

,OrgAirport,DayofYear,Hour,Flights
7077634,15454,365,23,1
7077635,10140,365,23,1
7077644,14843,365,23,1
7077648,14570,365,23,1
7077649,14524,365,20,1
7077651,11503,365,14,1
7077666,11503,365,13,1
7077667,14259,365,20,1
7077669,10372,365,18,1
7077670,15380,365,17,1


It would be good to check merging is successful. 

In [11]:
l0 = lookup.index.get_level_values(0)
l1 = lookup.index.get_level_values(1)
l2 = lookup.index.get_level_values(2)
cond = (l0 == 14057) & (l1 == 365) & (l2 == 23)
lookup[cond]

,,,Flights
OrgAirport,DayofYear,Hour,
14057,365,23,2


In [12]:
# Now, merge number of flights
n_flights = temp.merge(lookup, left_on=['OrgAirport', 'DayofYear', 'Hour'], right_index=True).iloc[:, -1]

In [13]:
feat['n_flights'] = n_flights
del lookup, temp

Another feature that needs to be extracted is holidays. In this case, We will be creating features that show how many days a particular flight is away from these federal holidays.

In [14]:
hld = holidays.US(years=2018, observed=False)
hld = pd.DataFrame.from_dict(hld, orient='index', columns=['Holidays'])
hld['Holidays'] = hld.Holidays.str.replace('[^\w\s]','').str.replace(' ', '')
hld.loc[datetime.date(2019,1,1)] = 'NxtNewYearsDay'
hld

,Holidays
2018-01-01,NewYearsDay
2018-01-15,MartinLutherKingJrDay
2018-02-19,WashingtonsBirthday
2018-05-28,MemorialDay
2018-07-04,IndependenceDay
2018-09-03,LaborDay
2018-10-08,ColumbusDay
2018-11-11,VeteransDay
2018-11-22,Thanksgiving
2018-12-25,ChristmasDay


In [15]:
for idx, row in hld.iterrows():
    feat[row.values[0]] = ((flights.ScDepTime - pd.to_datetime(idx)).astype('timedelta64[D]')).abs()
del hld

In [16]:
feat.head()

,month,weekDay,dayofMonth,dayofYear,hour,iata,tailNum,orgAirport,destAirport,temp,...,MartinLutherKingJrDay,WashingtonsBirthday,MemorialDay,IndependenceDay,LaborDay,ColumbusDay,VeteransDay,Thanksgiving,ChristmasDay,NxtNewYearsDay
0,1,1,1,1,0,UA,N14240,10299,11292,26.100000,...,14.0,49.0,147.0,184.0,245.0,280.0,314.0,325.0,358.0,365.0
1,1,1,1,1,0,AS,N530AS,10299,12889,26.100000,...,14.0,49.0,147.0,184.0,245.0,280.0,314.0,325.0,358.0,365.0
2,1,1,1,1,0,AA,N156UW,14771,11057,57.000000,...,14.0,49.0,147.0,184.0,245.0,280.0,314.0,325.0,358.0,365.0
3,1,1,1,1,0,DL,N669DN,14771,13487,57.000000,...,14.0,49.0,147.0,184.0,245.0,280.0,314.0,325.0,358.0,365.0
4,1,1,1,1,0,UA,N66808,14747,12266,37.900002,...,14.0,49.0,147.0,184.0,245.0,280.0,314.0,325.0,358.0,365.0


In [17]:
tail_nums = flights.TailNum.drop_duplicates()
tail_nums = tail_nums.reset_index(drop=True)

In [18]:
feat['prevArrDel'] = pd.NaT
feat['timeDiff'] = pd.NaT

In [20]:
bar = progressbar.ProgressBar(max_value=len(tail_nums))
for key, val in tail_nums.items():
    idx1 = flights.loc[flights.loc[:, 'TailNum'] == val].sort_values('DepTime').iloc[:-1, 1].index
    idx2 = flights.loc[flights.loc[:, 'TailNum'] == val].sort_values('DepTime').iloc[1:, 1].index
    feat.loc[idx2, 'prevArrDel'] = flights.loc[idx1, 'ArrDelay'].astype('timedelta64[m]').values
    feat.loc[idx2, 'timeDiff'] = (flights.loc[idx2, 'ScDepTime'] - flights.loc[idx1, 'ArrTime']).astype('timedelta64[m]')
    bar.update(key+1)

100% (5717 of 5717) |####################| Elapsed Time: 0:50:52 ETA:  00:00:00